Here is the code to release the queries with the parameters you have selected using the OpenDP library

In [ ]:
# notebook imports
import opendp.prelude as dp
import pandas as pd
import numpy as np
dp.enable_features("contrib")
from typing import List

## Query: age_average

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv')

In [ ]:
query_info = {"age_average": {"lower_bound": 0, "upper_bound": 120, "bins": 10, "column": "age", "epsilon": 0.1, "url": "https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv"}}

In [ ]:
domainType = dp.domain_of(List[int])

In [ ]:
context = dp.Context.compositor(
                                data=list(df["age"]),  # Actual column name
                                privacy_unit=dp.unit_of(contributions=1),
                                privacy_loss=dp.loss_of(epsilon=0.1),  # Actual epsilon
                                domain=domainType, 
                                split_evenly_over=1
                            )
dp_sum = context.query().clamp((0, 120)).sum().laplace()
private_release = dp_sum.release() / df.shape[0]  # Average calculation
print(f"Private release for query=age_average: {private_release}")